# Применение парафрейзера rut5-base-paraphraser
# на параллельном корпусе парафраз RuAdapt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import pandas as pd
# df = pd.read_csv('/content/drive/MyDrive/zlatoust_sentence_aligned_with_CATS.csv')
# df

,source,cos_sim,target,level
0,Ну что ж.,1.000000,Ну что ж.,b2
1,"— Ах, только не мужем, с простою усмешкой сказ...",0.863004,"— Ах, только не мужем, — с простою усмешкой ск...",b1
2,"Мне скоро шестьдесят, я старик, одинокий, ничт...",0.653318,"Ничего во мне нет хорошего, кроме этой любви к...",b2
3,Да!,0.093739,Обстановочка-то какова!,a2_b1
4,"Я вполне уверен, что вы раскаялись и раскаивае...",0.621839,"Я уверен, что вы раскаялись и будете содейство...",b1
...,...,...,...,...
24227,У нас в феврале масленица,0.960889,У нас в феврале масленица.,b1_c1
24228,— Разводиться мне или нет?,0.962737,— Разводиться мне или нет?,b1
24229,"Парусиновую куртку Петр Алексеевич сбросил, ру...",0.938603,"Парусиновую куртку Пётр Алексеевич сбросил, р...",b1_b2
24230,Ай!,1.000000,Ай!,b2


In [ ]:
# Создание датафрейма из 2000 случайно выбранных предложений
# new_df = df[(df['cos_sim'] >= 0.31) & (df['cos_sim'] <= 0.98)].drop(['level'], axis=1).sample(n = 2000)
# new_df

In [ ]:
# Конвертация в csv-файл
# new_df.to_csv("2000_random_rows.csv")

In [ ]:
import pandas as pd
new_df = pd.read_csv('/content/drive/MyDrive/2000_random_rows.csv')
new_df

,Unnamed: 0,source,cos_sim,target
0,15502,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...
1,23216,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о..."
2,20768,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...
3,1059,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё."
4,11600,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн..."
...,...,...,...,...
1995,13103,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч..."
1996,1225,— спросил он,0.568087,— спросил Сергей
1997,13123,Ударит первым.,0.382794,Начнёт бой первым.
1998,16923,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в..."


In [ ]:
pip install sentence-transformers

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
MODEL_NAME = 'cointegrated/rut5-base-paraphraser'
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

In [ ]:
def get_paraphrase(text, beams=5, grams=4, do_sample=False):
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    max_size = int(x.input_ids.shape[1] * 1.5 + 10)
    out = model.generate(**x, encoder_no_repeat_ngram_size=grams, num_beams=beams, max_length=max_size, do_sample=do_sample)
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [ ]:
import logging

In [ ]:
logging.getLogger("transformers").setLevel(logging.ERROR)

In [ ]:
%%time
new_df['output'] = new_df['source'].apply(get_paraphrase)

CPU times: user 2h 5min 15s, sys: 15.9 s, total: 2h 5min 31s
Wall time: 2h 5min 55s


In [ ]:
# Конвертация в csv-файл вместе с output
# new_df.to_csv("result_t5.csv")

# Оценка результата парафрейзера rut5-base-paraphraser
# с помощью дистрибутивной модели rubert-base-cased-sentence

In [ ]:
pip install sentence-transformers

In [ ]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

In [ ]:
import gc
import multiprocessing as mp
from tqdm import tqdm

In [ ]:
import torch
cos = torch.nn.CosineSimilarity(dim=1)

In [ ]:
%%time
DF_FILE = '/content/drive/MyDrive/result_t5.csv'
# размер одного обрабатываемого кусочка датафрейма
CHUNKSIZE = 20

def process_frame(new_df):
  # функция принимает на вход датафрейм и возвращает косинусные подобия для каждого ряда
  similarities_1 = []
  for n, row in tqdm(new_df.iterrows()):
    s = row['source']
    o = row['output']
    tokens_s = tokenizer([s], return_tensors="pt", padding=True)
    tokens_o = tokenizer([o], return_tensors="pt", padding=True)
    out_s = model(**tokens_s)
    out_o = model(**tokens_o)
    vec_s = torch.mean(out_s.last_hidden_state, dim=1)
    vec_o = torch.mean(out_o.last_hidden_state, dim=1)
    cos_sim = cos(vec_s, vec_o)
    # item() превращает тензор во float
    similarities_1.append(cos_sim.item())
    # на всякий случай собираем ненужное при помощи garbage collector
    gc.collect()
  return similarities_1

# читаем файл по кусочкам заданного размера
reader = pd.read_csv(DF_FILE, chunksize=CHUNKSIZE)
# количество процессов, которые могут идти одновременно
pool = mp.Pool(4)

funclist_1 = []
for new_df_1 in reader:
  # открываем и обрабатываем каждый кусочек
  new_df.columns = ['Unnamed:', 'source', 'cos_sim', 'target', 'output']
  f = pool.apply_async(process_frame, [new_df_1])
  funclist_1.append(f)

result_1 = []
for f in funclist_1:
  # таймаут можно увеличить, если возникает ошибка
  sims = f.get(timeout=500)
  result_1.extend(sims)

20it [00:26,  1.33s/it]
20it [00:27,  1.37s/it]
20it [00:27,  1.37s/it]
20it [00:28,  1.43s/it]
20it [00:23,  1.18s/it]
20it [00:25,  1.27s/it]
20it [00:28,  1.40s/it]
20it [00:27,  1.37s/it]
20it [00:25,  1.29s/it]
20it [00:27,  1.37s/it]
20it [00:25,  1.26s/it]
20it [00:29,  1.48s/it]
20it [00:28,  1.45s/it]
20it [00:28,  1.41s/it]
20it [00:35,  1.77s/it]
20it [00:28,  1.43s/it]
20it [00:24,  1.22s/it]
20it [00:28,  1.41s/it]
20it [00:28,  1.44s/it]
20it [00:26,  1.31s/it]
20it [00:21,  1.08s/it]
20it [00:24,  1.22s/it]
20it [00:27,  1.40s/it]
20it [00:28,  1.43s/it]
20it [00:24,  1.21s/it]
20it [00:22,  1.11s/it]
20it [00:25,  1.28s/it]
20it [00:23,  1.18s/it]
20it [00:26,  1.31s/it]
20it [00:24,  1.24s/it]
20it [00:25,  1.28s/it]
20it [00:25,  1.27s/it]
20it [00:24,  1.21s/it]
20it [00:25,  1.29s/it]
20it [00:25,  1.27s/it]
20it [00:25,  1.28s/it]
20it [00:23,  1.18s/it]
20it [00:24,  1.21s/it]
20it [00:24,  1.21s/it]
20it [00:24,  1.24s/it]
20it [00:24,  1.25s/it]
20it [00:23,  1.

CPU times: user 7.26 s, sys: 1.01 s, total: 8.27 s
Wall time: 10min 57s


In [ ]:
result_1[:2000]

[0.9805588722229004,
 0.9795916080474854,
 0.7931328415870667,
 0.9359561800956726,
 0.9582939147949219,
 0.9573853015899658,
 0.9769312143325806,
 0.8747115135192871,
 0.8534017205238342,
 0.9145289063453674,
 0.9781668782234192,
 0.9681917428970337,
 0.9551096558570862,
 0.947014570236206,
 0.9883411526679993,
 0.9683281779289246,
 0.9531217813491821,
 0.8189772367477417,
 0.8960195183753967,
 0.7097024917602539,
 0.9739128947257996,
 0.9437945485115051,
 0.9744280576705933,
 0.9640730023384094,
 0.9443436861038208,
 0.9307190179824829,
 0.9765483736991882,
 0.6464532613754272,
 0.9421797394752502,
 0.8593503832817078,
 0.9553813934326172,
 0.8968273401260376,
 0.9357262849807739,
 0.8736504316329956,
 0.9245757460594177,
 0.9337471127510071,
 0.9674961566925049,
 0.9632424712181091,
 0.9490456581115723,
 0.9144155383110046,
 0.9568389058113098,
 0.8481199145317078,
 0.9393123388290405,
 0.9451764822006226,
 0.9812790751457214,
 0.9490305781364441,
 0.9533652663230896,
 0.97311913967

In [ ]:
# Среднее значение
import numpy as np
np.mean(np.array(result_1))

0.9279100150316953

In [ ]:
# Среднеквадратичное отклонение
np.std(np.array(result_1))

0.06018995518214228

# METEOR
https://huggingface.co/spaces/evaluate-metric/meteor

In [ ]:
pip install evaluate

In [ ]:
from evaluate import load

meteor = load("meteor")

In [ ]:
import pandas as pd
final_df = pd.read_csv('/content/drive/MyDrive/result_t5.csv').drop("Unnamed: 0", axis=1)
final_df

,Unnamed: 0.1,source,cos_sim,target,output
0,0,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...,"Дочь моя по закону - не дочь моя, а дочь Карен..."
1,1,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о...","""""""Да, все кончено и опять я одна"""", - сказала..."
2,2,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...,На приглашении его двое старших сразу же пошли...
3,3,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё.",Все это случилось в один миг... Официант не по...
4,4,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн...","Отродившись, грешник, я так и не жил... Дом бо..."
...,...,...,...,...,...
1995,1995,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч...","Но нужно было идти, а он по первой поездке, но..."
1996,1996,— спросил он,0.568087,— спросил Сергей,— Он спросил его.
1997,1997,Ударит первым.,0.382794,Начнёт бой первым.,Ударит первый.
1998,1998,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в...","- Конечно, конечно... Я разовью мысль, стараюс..."


In [ ]:
predictions = final_df['output'].tolist()

In [ ]:
references = final_df['source'].tolist()

In [ ]:
results = meteor.compute(predictions=predictions, references=references)
results

{'meteor': 0.4217108846760342}

# BLEU
https://huggingface.co/spaces/evaluate-metric/bleu

In [ ]:
pip install evaluate

In [ ]:
from evaluate import load

bleu = load("bleu")

In [ ]:
import pandas as pd
final_df = pd.read_csv('/content/drive/MyDrive/result_t5.csv').drop("Unnamed: 0", axis=1)
final_df

,Unnamed: 0.1,source,cos_sim,target,output
0,15502,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...,"Дочь моя по закону - не дочь моя, а дочь Карен..."
1,23216,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о...","""""""Да, все кончено и опять я одна"""", - сказала..."
2,20768,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...,На приглашении его двое старших сразу же пошли...
3,1059,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё.",Все это случилось в один миг... Официант не по...
4,11600,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн...","Отродившись, грешник, я так и не жил... Дом бо..."
...,...,...,...,...,...
1995,13103,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч...","Но нужно было идти, а он по первой поездке, но..."
1996,1225,— спросил он,0.568087,— спросил Сергей,— Он спросил его.
1997,13123,Ударит первым.,0.382794,Начнёт бой первым.,Ударит первый.
1998,16923,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в...","- Конечно, конечно... Я разовью мысль, стараюс..."


In [ ]:
output = final_df['output'].tolist()

In [ ]:
source = final_df['source'].tolist()

In [ ]:
init = []

for word in source:
  init.append(word.split("''"))

init

In [ ]:
predictions = output
print(predictions)

references = init
print(references)

['Дочь моя по закону - не дочь моя, а дочь Каренины.', '"""Да, все кончено и опять я одна"", - сказала она сама и не сняв шляпы села на кресло, стоявшее на камине."', 'На приглашении его двое старших сразу же пошли к нему, и пошли с ним, как и побегали с ним няня, мисс Гуля или мать.', 'Все это случилось в один миг... Официант не помня о себе бросился к ней, около Наташи Филипповой не было свиты, приличный джентльмен среднего возраста уже не успел абсолютно стушеваться, а джентльмен навеселе смотрел в сторону и думал, что это было моча.', 'Отродившись, грешник, я так и не жил... Дом большой, казенный, а мне целая комната и кровать.', 'Но, приятель, не помнишь ли вы покойного Льва Николаевича?', 'Я думаю, он очень застенчивый...Когда мы с ним вдвоем, он очень умный, ласковый, и в обществе грубый, бретерный.', 'Она не плачет уже... она хорошая... Слушается глухой, далекой выстрелом.', 'Он так ненавидел блестящего Шарля Д’Эвре и так мечтал обойти его!', '— До свидания!', '– Как не узнать!

In [ ]:
bleu_results = bleu.compute(predictions=predictions, references=references)
print(bleu_results)

{'bleu': 0.04689093735791286, 'precisions': [0.5402519396012346, 0.1401902181914549, 0.016645286442852226, 0.0038348672802454315], 'brevity_penalty': 1.0, 'length_ratio': 1.0141857972812904, 'translation_length': 35961, 'reference_length': 35458}


# SacreBLEU
https://huggingface.co/spaces/evaluate-metric/sacrebleu

In [ ]:
pip install evaluate

In [ ]:
pip install sacrebleu

In [ ]:
from evaluate import load

sacrebleu = load("sacrebleu")

In [ ]:
import pandas as pd
final_df = pd.read_csv('/content/drive/MyDrive/result_t5.csv').drop("Unnamed: 0", axis=1)
final_df

,Unnamed: 0.1,source,cos_sim,target,output
0,15502,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...,"Дочь моя по закону - не дочь моя, а дочь Карен..."
1,23216,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о...","""""""Да, все кончено и опять я одна"""", - сказала..."
2,20768,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...,На приглашении его двое старших сразу же пошли...
3,1059,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё.",Все это случилось в один миг... Официант не по...
4,11600,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн...","Отродившись, грешник, я так и не жил... Дом бо..."
...,...,...,...,...,...
1995,13103,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч...","Но нужно было идти, а он по первой поездке, но..."
1996,1225,— спросил он,0.568087,— спросил Сергей,— Он спросил его.
1997,13123,Ударит первым.,0.382794,Начнёт бой первым.,Ударит первый.
1998,16923,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в...","- Конечно, конечно... Я разовью мысль, стараюс..."


In [ ]:
output = final_df['output'].tolist()

In [ ]:
source = final_df['source'].tolist()

In [ ]:
init = []

for word in source:
  init.append(word.split("''"))

init

In [ ]:
predictions = output
print(predictions)

references = init
print(references)

['Дочь моя по закону - не дочь моя, а дочь Каренины.', '"""Да, все кончено и опять я одна"", - сказала она сама и не сняв шляпы села на кресло, стоявшее на камине."', 'На приглашении его двое старших сразу же пошли к нему, и пошли с ним, как и побегали с ним няня, мисс Гуля или мать.', 'Все это случилось в один миг... Официант не помня о себе бросился к ней, около Наташи Филипповой не было свиты, приличный джентльмен среднего возраста уже не успел абсолютно стушеваться, а джентльмен навеселе смотрел в сторону и думал, что это было моча.', 'Отродившись, грешник, я так и не жил... Дом большой, казенный, а мне целая комната и кровать.', 'Но, приятель, не помнишь ли вы покойного Льва Николаевича?', 'Я думаю, он очень застенчивый...Когда мы с ним вдвоем, он очень умный, ласковый, и в обществе грубый, бретерный.', 'Она не плачет уже... она хорошая... Слушается глухой, далекой выстрелом.', 'Он так ненавидел блестящего Шарля Д’Эвре и так мечтал обойти его!', '— До свидания!', '– Как не узнать!

In [ ]:
sacrebleu_results = sacrebleu.compute(predictions=predictions, references=references)
print(sacrebleu_results)
print(round(sacrebleu_results["score"], 1))

{'score': 4.689093735791287, 'counts': [19428, 4761, 532, 115], 'totals': [35961, 33961, 31961, 29988], 'precisions': [54.02519396012347, 14.01902181914549, 1.6645286442852225, 0.38348672802454314], 'bp': 1.0, 'sys_len': 35961, 'ref_len': 35458}
4.7


# ROUGE
https://huggingface.co/spaces/evaluate-metric/rouge

In [ ]:
pip install evaluate

In [ ]:
pip install rouge_score

In [ ]:
from evaluate import load

rouge = load("rouge")

In [ ]:
import pandas as pd
final_df = pd.read_csv('/content/drive/MyDrive/result_t5.csv').drop("Unnamed: 0", axis=1)
final_df

,Unnamed: 0.1,source,cos_sim,target,output
0,15502,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...,"Дочь моя по закону - не дочь моя, а дочь Карен..."
1,23216,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о...","""""""Да, все кончено и опять я одна"""", - сказала..."
2,20768,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...,На приглашении его двое старших сразу же пошли...
3,1059,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё.",Все это случилось в один миг... Официант не по...
4,11600,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн...","Отродившись, грешник, я так и не жил... Дом бо..."
...,...,...,...,...,...
1995,13103,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч...","Но нужно было идти, а он по первой поездке, но..."
1996,1225,— спросил он,0.568087,— спросил Сергей,— Он спросил его.
1997,13123,Ударит первым.,0.382794,Начнёт бой первым.,Ударит первый.
1998,16923,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в...","- Конечно, конечно... Я разовью мысль, стараюс..."


In [ ]:
output = final_df['output'].tolist()

In [ ]:
source = final_df['source'].tolist()

In [ ]:
predictions = output
print(predictions)

references = source
print(references)

['Дочь моя по закону - не дочь моя, а дочь Каренины.', '"""Да, все кончено и опять я одна"", - сказала она сама и не сняв шляпы села на кресло, стоявшее на камине."', 'На приглашении его двое старших сразу же пошли к нему, и пошли с ним, как и побегали с ним няня, мисс Гуля или мать.', 'Все это случилось в один миг... Официант не помня о себе бросился к ней, около Наташи Филипповой не было свиты, приличный джентльмен среднего возраста уже не успел абсолютно стушеваться, а джентльмен навеселе смотрел в сторону и думал, что это было моча.', 'Отродившись, грешник, я так и не жил... Дом большой, казенный, а мне целая комната и кровать.', 'Но, приятель, не помнишь ли вы покойного Льва Николаевича?', 'Я думаю, он очень застенчивый...Когда мы с ним вдвоем, он очень умный, ласковый, и в обществе грубый, бретерный.', 'Она не плачет уже... она хорошая... Слушается глухой, далекой выстрелом.', 'Он так ненавидел блестящего Шарля Д’Эвре и так мечтал обойти его!', '— До свидания!', '– Как не узнать!

In [ ]:
rouge_results = rouge.compute(predictions=predictions, references=references, tokenizer=lambda x: x.split())
rouge_results

{'rouge1': 0.3658157491589037,
 'rouge2': 0.05977302738786553,
 'rougeL': 0.3434063500018344,
 'rougeLsum': 0.34343748437076155}

# BERTScore
https://huggingface.co/spaces/evaluate-metric/bertscore

In [ ]:
pip install evaluate

In [ ]:
pip install bert_score

In [ ]:
from evaluate import load

bertscore = load("bertscore")

In [ ]:
import pandas as pd
final_df = pd.read_csv('/content/drive/MyDrive/result_t5.csv').drop("Unnamed: 0", axis=1)
final_df

,Unnamed: 0.1,source,cos_sim,target,output
0,15502,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...,"Дочь моя по закону - не дочь моя, а дочь Карен..."
1,23216,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о...","""""""Да, все кончено и опять я одна"""", - сказала..."
2,20768,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...,На приглашении его двое старших сразу же пошли...
3,1059,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё.",Все это случилось в один миг... Официант не по...
4,11600,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн...","Отродившись, грешник, я так и не жил... Дом бо..."
...,...,...,...,...,...
1995,13103,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч...","Но нужно было идти, а он по первой поездке, но..."
1996,1225,— спросил он,0.568087,— спросил Сергей,— Он спросил его.
1997,13123,Ударит первым.,0.382794,Начнёт бой первым.,Ударит первый.
1998,16923,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в...","- Конечно, конечно... Я разовью мысль, стараюс..."


In [ ]:
output = final_df['output'].tolist()

In [ ]:
source = final_df['source'].tolist()

In [ ]:
predictions = output
print(predictions)

references = source
print(references)

['Дочь моя по закону - не дочь моя, а дочь Каренины.', '"""Да, все кончено и опять я одна"", - сказала она сама и не сняв шляпы села на кресло, стоявшее на камине."', 'На приглашении его двое старших сразу же пошли к нему, и пошли с ним, как и побегали с ним няня, мисс Гуля или мать.', 'Все это случилось в один миг... Официант не помня о себе бросился к ней, около Наташи Филипповой не было свиты, приличный джентльмен среднего возраста уже не успел абсолютно стушеваться, а джентльмен навеселе смотрел в сторону и думал, что это было моча.', 'Отродившись, грешник, я так и не жил... Дом большой, казенный, а мне целая комната и кровать.', 'Но, приятель, не помнишь ли вы покойного Льва Николаевича?', 'Я думаю, он очень застенчивый...Когда мы с ним вдвоем, он очень умный, ласковый, и в обществе грубый, бретерный.', 'Она не плачет уже... она хорошая... Слушается глухой, далекой выстрелом.', 'Он так ненавидел блестящего Шарля Д’Эвре и так мечтал обойти его!', '— До свидания!', '– Как не узнать!

In [ ]:
bertscore_results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")
bertscore_results

{'precision': [0.9650402069091797,
  0.9517402648925781,
  0.9470125436782837,
  0.9321717619895935,
  0.9388178586959839,
  0.9526749849319458,
  0.9428929090499878,
  0.922191858291626,
  0.9661402106285095,
  0.9825968742370605,
  0.9696567058563232,
  0.9325628280639648,
  0.9271388053894043,
  0.9457023739814758,
  0.9755760431289673,
  0.9700189232826233,
  0.9234797358512878,
  0.9416605830192566,
  0.9355577826499939,
  0.9282738566398621,
  0.9388116002082825,
  0.9282718896865845,
  0.9711083173751831,
  0.9457692503929138,
  0.9697477221488953,
  0.939485490322113,
  0.9570019245147705,
  0.9316984415054321,
  0.9441132545471191,
  0.948498547077179,
  0.9485751390457153,
  0.9397830963134766,
  0.9877111911773682,
  0.9263415336608887,
  0.9417446851730347,
  0.9425278902053833,
  0.955644965171814,
  0.9593693017959595,
  0.9407414793968201,
  0.9326826333999634,
  0.9388142228126526,
  0.9424519538879395,
  0.9587997198104858,
  0.9326419234275818,
  0.9538534879684448,
 

In [ ]:
import numpy as np
np.mean(np.array(bertscore_results['precision']))

0.9445786751806736

In [ ]:
import numpy as np
np.mean(np.array(bertscore_results['recall']))

0.9430376712381839

In [ ]:
import numpy as np
np.mean(np.array(bertscore_results['f1']))

0.9437244481444359

# ParaScore
https://github.com/shadowkiller33/ParaScore

In [ ]:
pip install parascore==1.0.5

In [ ]:
from parascore import ParaScorer

In [ ]:
scorer = ParaScorer(lang="ru", model_type='bert-base-uncased')

In [ ]:
import pandas as pd
final_df = pd.read_csv('/content/drive/MyDrive/result_t5.csv').drop("Unnamed: 0", axis=1)
final_df

,Unnamed: 0.1,source,cos_sim,target,output
0,15502,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...,"Дочь моя по закону - не дочь моя, а дочь Карен..."
1,23216,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о...","""""""Да, все кончено и опять я одна"""", - сказала..."
2,20768,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...,На приглашении его двое старших сразу же пошли...
3,1059,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё.",Все это случилось в один миг... Официант не по...
4,11600,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн...","Отродившись, грешник, я так и не жил... Дом бо..."
...,...,...,...,...,...
1995,13103,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч...","Но нужно было идти, а он по первой поездке, но..."
1996,1225,— спросил он,0.568087,— спросил Сергей,— Он спросил его.
1997,13123,Ударит первым.,0.382794,Начнёт бой первым.,Ударит первый.
1998,16923,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в...","- Конечно, конечно... Я разовью мысль, стараюс..."


In [ ]:
cands = final_df['output'].tolist()
sources = final_df['source'].tolist()
refs = final_df['target'].tolist()

In [ ]:
parascore_results = scorer.base_score(cands, sources, refs, batch_size=16)
parascore_results

[0.926669852733612,
 0.9163721966743469,
 0.8818584251403808,
 0.8665673899650573,
 0.8797760226636169,
 0.8838790225982666,
 0.8666337299346923,
 0.8597915339469909,
 0.9217603094761189,
 0.9419021838051932,
 0.9294330068996974,
 0.8829684424400329,
 0.8606477546691894,
 0.8803718852996826,
 0.9733968544006347,
 0.9110486498245827,
 0.8989343912260873,
 0.9524541610247129,
 0.8785649108886718,
 0.8445820236692623,
 0.8682317221342628,
 0.8950255299662496,
 0.9557556080818176,
 0.8708039889491797,
 0.9035727892603193,
 0.8632660675048828,
 0.9136027860641479,
 0.80413170337677,
 0.8623221087455749,
 0.8972525934783779,
 0.8789999055862426,
 0.9157641100883483,
 0.957727483340672,
 0.8674624133110046,
 0.8828274774551391,
 0.8551110792160034,
 0.9234997797012329,
 0.8815440987207874,
 0.8891412782669067,
 0.8809140491485595,
 0.8794200894995505,
 0.8706938862800598,
 0.9031301372391837,
 0.8635110425949096,
 0.8769440882162789,
 0.9407995751925877,
 0.9019098448753357,
 0.89623464822769

In [ ]:
import numpy as np
np.mean(np.array(parascore_results))

0.8850812885883945

In [ ]:
# Среднеквадратичное отклонение
np.std(np.array(parascore_results))

0.04337527357426147